#### Ejercicio 7

Escribir un código para estimar la norma infinito de una matriz, usar la fórmula cerrada. Cómo
serı́a un código si no tenemos una fórmula cerrada? Comparar.

##### Solución

Partiendo de la definición de norma infinito para una matriz a partir de la norma vectorial:

$$
\left\| A \right\|_{\infty} = \sup \left\{ \left\| Ax \right\|_{\infty} : \left\| x \right\|_{\infty} = 1 \right\}
$$

Y aplicando la fórmula cerrada para una matriz 
cuadrada de dimensión $n$ como:

$$
\left\| A \right\|_{\infty} = \max_{1 \le i \le n} \sum_{j=i}^n \left| a_{ij} \right|
$$

Podemos implementarla con la siguiente función `inf_norm_mat()`:

In [1]:
def inf_norm_mat(matrix):
    return max(
        sum(abs(elem) for elem in row) for row in matrix
    )

La verificamos para la matriz ejemplo de las notas de 
clase:

$$
\left( \begin{matrix} 1 & 1 \\ 3 & 0 \end{matrix} \right)
$$

En nuestro caso:

In [2]:
inf_norm_mat([[1,1],[3,0]])

3

En el caso de que no fuera posible obtener
la fórmula cerrada de nuestra norma (como sería
para el caso de una norma $p$ en general)
deberíamos aplicar un procedimiento estadístico
generando la mayor cantidad posible de vectores $x$
que posean norma 1. Definimo así una segunda versión
de nuestra función aprovechando que la función 
`matmul` del paquete `numpy` se comporta como
producto escalar si ambos parámetros son vectores
de una dimensión:

In [3]:
from random import randint
from numpy import matmul


def normalize_vec(vector, norm):
    return [elem / norm(vector) for elem in vector]


def inf_norm_vec(vector):
    return max(abs(elem) for elem in vector)


def inf_norm_mat2(matrix, sample_size):
    return max(
        inf_norm_vec(
            matmul(
                matrix, normalize_vec(vector, inf_norm_vec)
            )
        )
        for vector in (
            [
                randint(-100, 100)
                for _ in range(len(matrix[0]))
            ]
            for _ in range(sample_size)
        )
    )

Para determinadas matrices se puede obtener un resultado
aceptable con pocas muestras. Reutilizando nuestra matriz
de ejemplo:

In [4]:
inf_norm_mat2([[1,1],[3,0]], 10)

3.0

Sin embargo, otras matrices nos obligan a una mayor 
cantidad de muestra para objener un resultado 
cercano al de la fórmula cerrada. Con ayuda de nuestra
función auxiliar `table` que se lista en el Anexo, 
construiremos una tabla comparativa para los 
resultados obtenidos con una matriz en el cálculo
de su norma infinita con diferentes tamaños de 
muestra aleatorias de vectores de norma infinita 1:

In [5]:
from auxiliares import tabla

tabla(
    [10 ** x for x in range(6)],
    [
        "Tamaño de muestra",
        "$\\begin{Vmatrix}A\end{Vmatrix}_{\infty}$",
    ],
    [
        repr,
        lambda x: inf_norm_mat2([[1, -2, 5], [1, 2, 4]], x),
    ],
)

|Tamaño de muestra|$\begin{Vmatrix}A\end{Vmatrix}_{\infty}$|
|:-:|:-:
|1|5.78125|
|10|6.136363636363636|
|100|7.216216216216217|
|1000|7.901234567901234|
|10000|7.984615384615385|
|100000|8.0|


#### Ejercicio 11

Escribir un código para resolver el sistema matricial $Ax = b$ donde $A$ es una matriz que es una
permutación de una matríz triangular superior.

##### Solución

Asumiendo que la matríz $A$ es una permutación de una matriz triangular
superior, aplicamos el algoritmo de resolución de una tal matriz para 
un vector de valores independientes, siguiendo un ordenamiento dado 
por una matriz de permutaciones $P$. Es decir, que resolvemos la matriz
triangular superior $P \times A$ para el vector de valores independientes
$P \times b$. Así, la implementación de la función `solve_triu` resulta:

In [6]:
from itertools import takewhile
from functools import reduce


def solve_triu(matrix, values, permut):
    n = len(matrix)
    perm = []
    for row in permut:
        for i, elem in enumerate(row):
            if elem == 1:
                perm.append(i) 
    return reduce(
        lambda r, i: [
            (
                values[perm[i]]
                - matmul(
                    matrix[perm[i]][i + 1 : n],
                    r,
                )
            )
            / matrix[perm[i]][i]
        ]
        + r,
        reversed(range(n)),
        [],
    )

Verifiquemos la función con el siguiente sistema:

$$\left(\begin{matrix} 1 & 2 & 3 \\ 0 & 5 & 4 \\ 0 & 0 & 6 \end{matrix}\right) \times \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right) = \left(\begin{matrix} 7 \\ 8 \\ 9 \end{matrix}\right)$$

En código:

In [7]:
A = [[1,2,3],
     [0,5,4],
     [0,0,6]]
b = [7,8,9]
P = [[1,0,0],
     [0,1,0],
     [0,0,1]]
x = solve_triu(A, b, P)
print(x)

[1.7000000000000002, 0.4, 1.5]


Si nuestra solución es correcta se debe verificar que $P \times A \times x = P \times b$. 
En nuestro ejemplo aprovechamos que la matriz $P$ es la identidad, por lo que la 
verificación de la igualdad en código es:

In [8]:
matmul(A, solve_triu(A, b, P))

array([7., 8., 9.])

#### Ejercicio 14

Usar el proceso de eliminación de Gauss Escalado para encontrar la descomposición $P \cdot A = L \cdot U$ (comparar con Python) y resolver en cada uno de los casos.

$$\left(\begin{matrix} -1 & 1 & -4 \\ 2 & 2 & 0 \\ 3 & 3 & 2 \end{matrix}\right) \times \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right) = \left(\begin{matrix} 0 \\ 1 \\ \frac{1}{2} \end{matrix}\right)$$

$$\left(\begin{matrix} 1 & 6 & 0 \\ 2 & 1 & 0 \\ 0 & 2 & 1 \end{matrix}\right) \times \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right) = \left(\begin{matrix} 3 \\ 1 \\ 1 \end{matrix}\right)$$

##### Solución

Primero definimos nuestra implementación de descomposición $LU$ 
mediante el método de Gauss Escalado para una matriz cuadrada, de manera que nuestra 
función `lu` devolverá una matriz de permutaciones $P$ y una matriz 
que corresponde a la suma matriz triangular superior $U$ permutada 
más una matriz triangular inferior $L$ menos la matriz identidad, 
también permutada. Es decir que para mejor aprovechamiento
de memoria resolveremos la descomposición _in situ_ y nuestro 
resultado equivaldrá a $P^{-1} \times (L - I + U)$. Para reconstruir
nuestras matrices no hace falta calcular la inversa de la permutación
ya que es ella misma. La implementación de la función es:

In [9]:
def lu(matrix):
    n = len(matrix)
    permut = []
    rows = set(range(n))
    maxim = [max(row, key=abs) for row in matrix]
    for i in range(n):
        pivot = max(
            rows,
            key=lambda j: abs(matrix[j][i] / maxim[j]),
        )
        rows.remove(pivot)
        permut.append(pivot)
        for k in rows:
            matrix[k][i] /= matrix[pivot][i]
            for l in range(i + 1, n):
                matrix[k][l] -= (
                    matrix[k][i] * matrix[pivot][l]
                )
    return (
        [
            [1 if i == p else 0 for i in range(n)]
            for p in permut
        ],
        matrix
    )

De manera que si la intención no es resolver un
sistema sino presentar las matrices correspondientes,
podremos utilizar funciones de copias de matrices
del paquete `numpy`. En nuestro caso:

In [10]:
from numpy import triu, tril, identity


def split_lu(matrix):
    P, G1 = lu(matrix)
    G2 = matmul(P, G1)
    return P, tril(G2, -1) + identity(len(G2)), triu(G2)

Compararemos nuestros resultados con los de la implementación de
la descomposición que realiza el paquete `scipy` de Python:

In [11]:
from scipy.linalg import lu as lu2
P1, L1, U1 = split_lu([[-1,1,-4],[2,2,0],[3,3,2]])
P2, L2, U2 = lu2([[-1,1,-4],[2,2,0],[3,3,2]])
print(P1, L1, U1, sep="\n", end="\n\n")
print(P2, L2, U2, sep="\n")

[[0, 1, 0], [1, 0, 0], [0, 0, 1]]
[[ 1.   0.   0. ]
 [-0.5  1.   0. ]
 [ 1.5  0.   1. ]]
[[ 2.  2.  0.]
 [ 0.  2. -4.]
 [ 0.  0.  2.]]

[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
[[ 1.          0.          0.        ]
 [-0.33333333  1.          0.        ]
 [ 0.66666667  0.          1.        ]]
[[ 3.          3.          2.        ]
 [ 0.          2.         -3.33333333]
 [ 0.          0.         -1.33333333]]


Como podemos observar, las descomposiciones
son diferentes, lo que es esperable ya que
también difieren los pivotes elegidos. Puesto
que si existe una descomposición $LU$ entonces
existen infinitas, debemos verificar que ambas 
cumplan con la igualdad $A = P^{-1}LU$, teniendo
en cuenta que la inversa de la matriz permutaciones
es la propia matriz de permutaciones:

In [12]:
print(matmul(P1, matmul(L1,U1)), end="\n\n")
print(matmul(P2, matmul(L2,U2)))

[[-1.  1. -4.]
 [ 2.  2.  0.]
 [ 3.  3.  2.]]

[[-1.  1. -4.]
 [ 2.  2.  0.]
 [ 3.  3.  2.]]


Para resolver nuestros sistemas sólo nos falta resolver
el caso de una matriz triangular inferior. Su algoritmo
es muy simular al de la matriz triangular superior con
la salvedad de que en este caso asumiremos que la 
diagonal es la identidad a menos que se indique lo
contrario, mecanismo habitual para aprovechar la 
descomposición _in situ_. Veamos su implementación:

In [13]:
def solve_tril(matrix, values, permut, identity=True):
    n = len(matrix)
    perm = []
    for row in permut:
        for i, elem in enumerate(row):
            if elem == 1:
                perm.append(i) 
    return reduce(
        lambda r, i: r + [
            (
                values[perm[i]]
                - matmul(
                    matrix[perm[i]][:i],
                    r,
                )
            )
            / (1 if identity else matrix[perm[i]][i])
        ],
        range(n),
        [],
    )

Verifiquemos su funcionamiento con un ejemplo
en espejo al utilizado al verificar el 
algoritmo para solucionar la matriz triangular superior:

$$\left(\begin{matrix} 6 & 0 & 0 \\ 4 & 5 & 0 \\ 3 & 2 & 1 \end{matrix}\right) \times \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right) = \left(\begin{matrix} 9 \\ 8 \\ 7 \end{matrix}\right)$$

En código:

In [14]:
A = [[6,0,0],
     [4,5,0],
     [3,2,1]]
b = [9,8,7]
P = [[1,0,0],
     [0,1,0],
     [0,0,1]]
solve_tril(A, b, P, identity=False)

[1.5, 0.4, 1.7000000000000002]

Si nuestra solución es correcta se debe verificar que $P \times A \times x = P \times b$. 
Nuevamente, en nuestro ejemplo aprovechamos que la matriz $P$ es la identidad, por lo que la 
verificación de la igualdad en código es:

In [15]:
matmul(A, solve_tril(A, b, P, identity=False))

array([9., 8., 7.])

Ahora estamos en condiciones de resolver nuestro
primer sistema:

$$\left(\begin{matrix} -1 & 1 & -4 \\ 2 & 2 & 0 \\ 3 & 3 & 2 \end{matrix}\right) \times \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right) = \left(\begin{matrix} 0 \\ 1 \\ \frac{1}{2} \end{matrix}\right)$$

En código:

In [16]:
P, L, U = split_lu([[-1,1,-4],[2,2,0],[3,3,2]])
I = identity(3)
x = solve_triu(U, solve_tril(L, matmul(P,[0, 1, 1/2]), I), I)
print(x)

[1.25, -0.75, -0.5]


Verifiquemos el resultado ya que $A \times x = b $:

In [17]:
matmul([[-1,1,-4],[2,2,0],[3,3,2]], x)

array([0. , 1. , 0.5])

En el caso que queramos trabajar con las matrices
_in situ_, será necesario permutar el resultado
intermedio en vez del vector de valores 
independientes, de modo que:

In [18]:
P, G = lu([[-1,1,-4],[2,2,0],[3,3,2]])
x = solve_triu(G, matmul(P, solve_tril(G, [0, 1, 1/2], P)), P)
print(x)

[1.25, -0.75, -0.5]


Usaremos este método para el siguiente sistema:

$$\left(\begin{matrix} 1 & 6 & 0 \\ 2 & 1 & 0 \\ 0 & 2 & 1 \end{matrix}\right) \times \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right) = \left(\begin{matrix} 3 \\ 1 \\ 1 \end{matrix}\right)$$

En código:

In [19]:
P, G = lu([[1,6,0],[2,1,0],[0,2,1]])
x = solve_triu(G, matmul(P, solve_tril(G, [3, 1, 1], P)), P)
print(x)

[-0.46590909090909094, 0.6818181818181819, -0.36363636363636365]


Verificamos el resultado:

In [20]:
matmul([[1,6,0],[2,1,0],[0,2,1]], x)

array([ 3.625, -0.25 ,  1.   ])

Como se puede ver, hemos obtenido una solición con
un error considerable con respecto a la deseado.
Verifiquemos con las librerías del lenguaje

In [21]:
P, L, U = lu2([[1,6,0],[2,1,0],[0,2,1]])
x = solve_triu(U, solve_tril(L, matmul(P,[3, 1, 1]), I), I)
print(x)

[0.2727272727272727, 0.45454545454545453, 0.09090909090909083]


Finalmente, comprobemos cuan cerca de la solución está:

In [22]:
matmul([[1,6,0],[2,1,0],[0,2,1]], x)

array([3., 1., 1.])

#### Ejercicio 15

Calcular los primeros 100 términos del método de Richardson para el sistema

$$
    \left(\begin{matrix} 1 & 1/2 & 1/3 \\ 1/3 & 1 & 1/2 \\ 1/2 & 1/3 & 1 \end{matrix}\right) 
    \times 
    \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right)
    = 
    \left(\begin{matrix} 11/18 \\ 11/18 \\ 11/18 \end{matrix}\right)
$$

##### Solución

Sabemos que el método de Richardson pertenece
a la familia de métodos iterativos tal que:

$$ Q \times x_{k} = (Q - A) \times x_{k-1} + b $$

En el caso del método de Richardson, $Q = I$, por lo
que nuestra fórmula iterativa resulta:

$$ x_{k} = (I - A) \times x_{k-1} + b $$

Si tomamos arbitrariamente que $x_0 = 0$ entonces $x_1 = b$, que es desde dónde comenzaremos la iteración en
caso de que no se provea una solución inicial. Nuestra implementación resulta:

In [23]:
def richardson(matrix, values, terms, guess=None):
    results = []
    if not guess:
        guess = values
    matrix = identity(len(matrix)) - matrix 
    for _ in range(terms):
        results.append(guess)
        guess = matmul(matrix, guess) + values 
    return results

Resolvemos ahora el ejercicio con el siguiente código:

In [24]:
r = richardson(
        [[1, 1/2, 1/3],
         [1/3, 1, 1/2],
         [1/2, 1/3, 1]], 
        [11/18,
         11/18,
         11/18],
        100
    )
print(*r, sep="\n")

[0.6111111111111112, 0.6111111111111112, 0.6111111111111112]
[0.10185185 0.10185185 0.10185185]
[0.52623457 0.52623457 0.52623457]
[0.1725823 0.1725823 0.1725823]
[0.46729252 0.46729252 0.46729252]
[0.22170067 0.22170067 0.22170067]
[0.42636055 0.42636055 0.42636055]
[0.25581065 0.25581065 0.25581065]
[0.39793557 0.39793557 0.39793557]
[0.27949814 0.27949814 0.27949814]
[0.378196 0.378196 0.378196]
[0.29594778 0.29594778 0.29594778]
[0.36448796 0.36448796 0.36448796]
[0.30737114 0.30737114 0.30737114]
[0.35496849 0.35496849 0.35496849]
[0.31530404 0.31530404 0.31530404]
[0.34835775 0.34835775 0.34835775]
[0.32081299 0.32081299 0.32081299]
[0.34376695 0.34376695 0.34376695]
[0.32463865 0.32463865 0.32463865]
[0.3405789 0.3405789 0.3405789]
[0.32729536 0.32729536 0.32729536]
[0.33836498 0.33836498 0.33836498]
[0.32914029 0.32914029 0.32914029]
[0.33682753 0.33682753 0.33682753]
[0.3304215 0.3304215 0.3304215]
[0.33575986 0.33575986 0.33575986]
[0.33131123 0.33131123 0.33131123]
[0.335018

Comprobemos la exactitud de la solución:

In [25]:
matmul(
    [[1, 1/2, 1/3],
     [1/3, 1, 1/2],
     [1/2, 1/3, 1]], 
    r[-1]
)

array([0.6111111, 0.6111111, 0.6111111])

#### Ejercicio 16

Escribir un algoritmo para calcular los primeros M pasos del método de Jacobi, y Gauss-Seidel.

##### Solución

Para el caso del método de Jacobi, sabemos que
pertenece a la misma familia que la del método
de Richardson, sólo que en este caso la matriz
$Q$ es la matriz diagonal de $A$ por lo que su 
inversa es fácilmente calculable:

$$ Q^{-1}_{ij} = \left\{ \begin{matrix} 
        \frac{1}{A_{ij}} \quad \text{si} \quad i = j \\
        0 \quad \text{si} \quad i \ne j
                    \end{matrix}
            \right.
$$

Por lo que nuestra fórmula iterativa resulta:

$$ x_{k} = (I - Q^{-1}A) \times x_{k-1} + Q^{-1} \times b $$

Con $G = (I - Q^{-1}A)$ definida como:

$$
G_{ij} = \left\{\begin{matrix}
            0 \quad \text{si} \quad i = j \\
            - a_{ij} / a_{ii} \quad \text{si} \quad i \ne j
        \end{matrix}\right.
$$

Nuestra implementación resulta entonces:

In [26]:
def jacobi(matrix, values, terms, guess=None):
    results = []
    if not guess:
        guess = values
    for i, row in enumerate(matrix):
        diag = row[i]
        values[i] /= diag
        for j, elem in enumerate(row):
            matrix[i][j] = 0 if i == j else - elem / diag
    for _ in range(terms):
        results.append(guess)
        guess = matmul(matrix, guess) + values 
    return results

En el caso del método de Gauss-Seidel, la matriz
$Q$ resulta ser la tringular inferior de $A$, pero
como en este caso el cálculo de su inversa podría
resultar costoso, utilizaremos la fórmula iterativa:

$$ Q \times x_{k} = (Q - A) \times x_{k-1} + b $$

Donde $Q - A$ resulta ser la opuesta aditiva de
la triangular superior de $A$ menos los elementos 
de la diagonal, de manera que
nos queda por resolver un sistema triangular 
inferior. Su implementación:

In [27]:
def gauss_siedel(matrix, values, terms, guess=None):
    results = []
    if not guess:
        guess = values
    trian = tril(matrix)
    matrix = -triu(matrix, 1) 
    for _ in range(terms):
        guess = solve_tril(
            trian, 
            matmul(matrix, guess) + values,
            identity(len(trian)),
            identity=False
        )
        results.append(guess)
    return results

#### Ejercicio 17

Para el siguiente sistema mostrar que tanto Gauss-Seidel como Jacobi convergen para cualquier valor
inicial. Usar el ı́tem anterior para estimar la solución.

$$
    \left(\begin{matrix} 2 & -1 & 0 \\ 1 & 6 & -2 \\ 4 & -3 & 8 \end{matrix}\right) 
    \times 
    \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right)
    = 
    \left(\begin{matrix} 2 \\ -4 \\ 5 \end{matrix}\right)
$$

##### Solución

Mostramos a continuación como la solución para 
la matriz del ejercicio converge tanto en el
caso del método de Gauss-Seidel como el de 
Jacobi por ser ésta diagonal dominante:

Finalmente estimamos la solución para los
primero 50 términos de cada método y los
presentamos en la siguiente tabla:

In [31]:
j = jacobi([[2,-1,0],[1,6,-2],[4,-3,8]], [2,-4,5], 11)
g = gauss_siedel([[2,-1,0],[1,6,-2],[4,-3,8]], [2,-4,5], 11)
tabla(
    zip(range(11), j, g),
    [
        "n",
        "Jacobi $x_n$",
        "Gauss-Siedel $x_n$"
    ]
)

|n|Jacobi $x_n$|Gauss-Siedel $x_n$|
|:-:|:-:|:-:
|0|[1.0, -0.6666666666666666, 0.625]|[-1.0, 1.1666666666666667, 1.5625]|
|1|[ 0.66666667 -0.625      -0.125     ]|[1.5833333333333335, -0.40972222222222227, -0.3203125000000001]|
|2|[ 0.6875     -0.81944444  0.05729167]|[0.7951388888888888, -0.9059606481481483, -0.1123046875]|
|3|[ 0.59027778 -0.76215278 -0.02604167]|[0.5470196759259258, -0.7952715084876543, 0.05326334635416674]|
|4|[ 0.61892361 -0.77372685  0.04405382]|[0.6023642457561729, -0.7493062588413065, 0.04282803005642366]|
|5|[ 0.61313657 -0.755136    0.02539062]|[0.6253468705793468, -0.7566151350777499, 0.028595889056170387]|
|6|[ 0.622432   -0.76039255  0.03525571]|[0.621692432461125, -0.7607501090581307, 0.02887249287263849]|
|7|[ 0.61980372 -0.75865343  0.02863679]|[0.6196249454709346, -0.7603133266209429, 0.030070029781679164]|
|8|[ 0.62067329 -0.76042169  0.0306031 ]|[0.6198433366895286, -0.759950546187695, 0.030096876834850073]|
|9|[ 0.61978915 -0.75991118  0.02950522]|[0.6200247269061525, -0.7599718288727422, 0.02999820071964543]|
|10|[ 0.62004441 -0.76012978  0.03013873]|[0.6200140855636289, -0.7600029473540563, 0.02999185196041443]|


Verificamos como para el caso del método de Jacobi
la solución es ya de tres cifras significativas
ya en el onceavo término:

In [32]:
m = [[2,-1,0],[1,6,-2],[4,-3,8]]
matmul(m, j[-1])

array([ 2.0002186 , -4.00101176,  5.00167683])

Hacemos lo mismo con el método de Gauss-Siedel:

In [33]:
matmul(m, g[-1])

array([ 2.00003112, -3.9999873 ,  5.        ])